<a href="https://colab.research.google.com/github/navyasripenmetsa/CSL2050_PRML_Major_Course_Project/blob/Decision_Trees/Decision_Tree_HOG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

In [25]:
# 1. Load your CSV files
train_data = pd.read_csv('HOG_Training.csv')
test_data = pd.read_csv('HOG_Testing.csv')


In [26]:
X_train = train_data.iloc[:, 1:]  # All columns except the first one
y_train = train_data.iloc[:, 0]   # First column as target
X_test = test_data.iloc[:, 1:]    # All columns except the first one
y_test = test_data.iloc[:, 0]     # First column as target


In [27]:
test_mask = y_test.isin(y_train.unique())
X_test_filtered = X_test[test_mask]
y_test_filtered = y_test[test_mask]


In [28]:
# 4. Encode the class labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test_filtered)

In [29]:
# 5. Handle missing values (NaN) using imputation
print("Handling missing values...")
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test_filtered)

Handling missing values...
